In [12]:
# dependencies

import simplejson as json  
import os
import lzma
import re
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
pd.set_option('display.max_columns', None)
nlp = spacy.load('en')
import timeit


In [1]:
# working directory
%cd "C:\Users\Dolomite\Box\Riverstone 2 (Phoebe, Nick, Jared, John)\Cases (json files)\New_Hampshire"
%pwd

C:\Users\Dolomite\Box\Riverstone 2 (Phoebe, Nick, Jared, John)\Cases (json files)\New_Hampshire


'C:\\Users\\Dolomite\\Box\\Riverstone 2 (Phoebe, Nick, Jared, John)\\Cases (json files)\\New_Hampshire'

In [58]:
#Loading data
cases = []
 
with lzma.open('data.jsonl.xz') as infile:
    for line in infile:
        #decode the file into a convenient format
        record = json.loads(str(line, 'utf-8'))
        #if the decision date on the case matches one we're interested in, add to our list
        if (int(record['decision_date'].split("-")[0])>1900):
            cases.append(record)

print("Number of Cases: {}".format(len(cases)))

Number of Cases: 13190


In [15]:
#Extracting meta data and text of cases
def parse_cases(case):
    
    elem_dict = {}
    try:
        elem_dict["id"] = case['id']
    except IndexError:
        elem_dict["id"] = np.nan
    try:
        elem_dict["name"] = case['name']
    except IndexError:
        elem_dict["name"] = np.nan
    try:   
        elem_dict["name_abbreviation"] = case['name_abbreviation']
    except IndexError:
        elem_dict["name_abbreviation"] = np.nan
    try:
        elem_dict["parties"] = case['casebody']['data']['parties']
    except IndexError:
         elem_dict["parties"] = np.nan
    try:
        elem_dict["plaintiff"] = case['name_abbreviation'].split('v.')[0].strip()
    except IndexError:
        elem_dict["plaintiff"] = np.nan
    try:
        elem_dict["defendant"] = case['name_abbreviation'].split('v.')[1].strip()
    except IndexError:
        elem_dict["defendant"] = np.nan
    try:
        elem_dict["decision_date"] = case['decision_date']
    except IndexError:
        elem_dict["decision_date"] = np.nan
    try:
        elem_dict["docket_number"] = case['docket_number']
    except IndexError:
        elem_dict["docket_number"] = np.nan
        
    citation = [citation for citation in case['citations'] if citation['type'] == 'official'][0]
    elem_dict["citation_type"] = citation['type']
    elem_dict["citation_num"] = citation['cite']
    try:
        elem_dict["reporter"] = case['reporter']['full_name']
    except IndexError:
        elem_dict["reporter"] = np.nan
    try:
        elem_dict["court_id"] = case['court']['id']
    except IndexError:
        elem_dict["court_id"] = np.nan
    elem_dict["court_name"] = case['court']['name']
    elem_dict["jurisdiction_id"] = case['jurisdiction']['id']
    elem_dict["jurisdiction_name"] = case['jurisdiction']['name_long']
    try:
        elem_dict["plaintiff_attorneys"] = case['casebody']['data']['attorneys'][0]
    except IndexError:
        elem_dict["plaintiff_attorneys"] = np.nan
    try:
        elem_dict["defendants_attorneys"] = case['casebody']['data']['attorneys'][1]
    except IndexError:
        elem_dict["defendants_attorneys"] = np.nan
    elem_dict["judges"] = case['casebody']['data']['judges']
    opinion_type = [opinion['type'] for opinion in case['casebody']['data']['opinions']]
    elem_dict["opinion_type"]= opinion_type
    opinion_author = [opinion['author'] for opinion in case['casebody']['data']['opinions']]
    elem_dict["opinion_author"] = opinion_author
    elem_dict["head_matter"] = case['casebody']['data']['head_matter']
    opinion_text = [opinion['text'] for opinion in case['casebody']['data']['opinions']]
    elem_dict["opinion_text"] = opinion_text
    
    return elem_dict

In [60]:

#parsing the cases
parsed_cases = []
for case in cases:
    doc = parse_cases(case)
    parsed_cases.append(doc)
    
cases_df = pd.DataFrame(parsed_cases)
cases_df.head()


,citation_num,citation_type,court_id,court_name,decision_date,defendant,defendants_attorneys,docket_number,head_matter,id,judges,jurisdiction_id,jurisdiction_name,name,name_abbreviation,opinion_author,opinion_text,opinion_type,parties,plaintiff,plaintiff_attorneys,reporter
0,144 N.H. 131,official,8797,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\nFeld’s Case\nAugust 30, 1999\nT...",105092,"[THAYER, HORTON, and BRODERICK, JJ., did not s...",47,New Hampshire,Feld’s Case,Feld’s Case,"[JOHNSON, J.]","[JOHNSON, J.\nThe Supreme Court Committee on P...",[majority],[Feld’s Case],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports
1,144 N.H. 13,official,8797,New Hampshire Supreme Court,1999-07-13,Bennett,"Janice S. Peterson, public defender, of Keene,...",No. 97-387,No. 97-387\nThe State of New Hampshire v. Eric...,105095,"[THAYER, J., with whom HORTON, J., joined, con...",47,New Hampshire,The State of New Hampshire v. Eric Bennett,State v. Bennett,"[BRODERICK, j., THAYER, J.,]","[BRODERICK, j.\nAfter a jury trial in Superior...","[majority, concurrence]",[The State of New Hampshire v. Eric Bennett],State,"Philip T. McLaughlin, attorney general (John C...",New Hampshire Reports
2,144 N.H. 138,official,8797,New Hampshire Supreme Court,1999-08-30,NaN,"Shaines & McEachern, P.A., of Portsmouth (Paul...",No. LD-97-008,"No. LD-97-008\nRoberge’s Case\nAugust 30, 1999...",105099,"[THAYER, HORTON, and BRODERICK, JJ., did not s...",47,New Hampshire,Roberge’s Case,Roberge’s Case,[PER CURIAM.],[MEMORANDUM OPINION\nPER CURIAM.\nThe Supreme ...,[majority],[Roberge’s Case],Roberge’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports
3,144 N.H. 107,official,8797,New Hampshire Supreme Court,1999-08-03,NaN,"Wiggin & Nourie, P.A., of Manchester (Scott A....",No. 97-399,"No. 97-399\nAppeal of Donald E. Savage, Jr. (N...",105102,[All concurred.],47,New Hampshire,"Appeal of Donald E. Savage, Jr. (New Hampshire...",Appeal of Savage,"[BRODERICK, J.]","[BRODERICK, J.\nThe petitioner, Donald E. Sava...",[majority],"[Appeal of Donald E. Savage, Jr. (New Hampshir...",Appeal of Savage,"Fitzgerald, & Sessler, P.A., of Laconia (Shawn...",New Hampshire Reports
4,144 N.H. 44,official,8797,New Hampshire Supreme Court,1999-07-21,NaN,"Philip T. McLaughlin, attorney general (Dougla...",No. 97-522,"No. 97-522\nAppeal of William H. Morgan, R.PH....",105108,[],47,New Hampshire,"Appeal of William H. Morgan, R.PH. (New Hampsh...",Appeal of Morgan,"[HORTON, J.]","[HORTON, J.\nThe petitioner, William H. Morgan...",[majority],"[Appeal of William H. Morgan, R.PH. (New Hamps...",Appeal of Morgan,"William, II. Loftus, P.C., of Lebanon (William...",New Hampshire Reports


In [2]:
# Create an empty column for the decisions
cases_df['decision']=""
# cases_df.head()

In [14]:
# Obtaining the opinion decision. This takes a long time to run

#counter
n=0

#timer
start = timeit.default_timer()
print(start)

# for some reason the two loops worked faster than one of for i in range(len(cases_df))
for num in range(27):
    for i in range((num*500),(num*500+500)):
        
        # create a spacy doc of the opinion text. The majority decision is always written first
        try:
            doc = nlp(cases_df['opinion_text'][i][0])
        except:
            # sometimes the opinion text is a string and not a list
            try:
                doc = nlp(cases_df['opinion_text'][i])
            except:
                pass
        n+=1

        # Within the last 15, 25, 35 etc. tokens search for a new line and take everything after that, ie, the last paragraph
        # the size increases so as to not grab 2 small entries. If a line break \n isn't found then 
        # look in the next larger group of tokens.
        decision = (re.findall("\n(.*)", doc[-15:-1].text, flags=re.IGNORECASE))
        if len(decision)==0:
            decision = (re.findall("\n(.*)", doc[-25:-1].text, flags=re.IGNORECASE))
            if len(decision)==0:
                decision = (re.findall("\n(.*)", doc[-35:-1].text, flags=re.IGNORECASE))
                if len(decision)==0:
                    decision = (re.findall("\n(.*)", doc[-50:-1].text, flags=re.IGNORECASE))
        
        # define the decision for that particular row, i
        cases_df.at[i,'decision']=decision
        
        # something to print to show progress
        if n%100==0:
            print(n, "case#:",i)
            
            #counter
            stop = timeit.default_timer()
            print(stop-start)
    


11118.626594792
100 case#: 13099
107.57082850599909
200 case#: 13199
178.77926745200057
300 case#: 13299
182.2947366190001
400 case#: 13399
185.39106710799933
500 case#: 13499
188.4030755040003
600 case#: 13599
191.72607041099945
700 case#: 13699
194.75362320900058
800 case#: 13799
197.9328401849998
900 case#: 13899
201.2903494429993


In [ ]:
# Create a criterion to subset the opinion_type that is 'majority' only
criterion = cases_df['opinion_type'].map(lambda x: x in [['majority']])

# create the new dataframe with the criterion
majority_cases =cases_df[criterion]

In [ ]:
# BELOW HERE IS JUST EXPLORATION

In [62]:
# You can see the frequency of judges or opinion_author

pd.set_option('display.max_rows', None)

cases_df['judges'].value_counts()
cases_df['opinion_author'].value_counts()

[None]                                                                                                        532
[Per curiam.]                                                                                                 519
[Duncan, J.]                                                                                                  499
[Lampron, J.]                                                                                                 417
[Batchelder, J.]                                                                                              334
[Parsons, C. J.]                                                                                              294
[Kenison, C. J.]                                                                                              276
[Walker, J.]                                                                                                  273
[Young, J.]                                                                             

In [57]:
n=0
for i in range(len(cases_df)):
    if len(cases_df['judges'][i])==1:
        print(i, cases_df['judges'][i])
        print(i, cases_df['opinion_author'][i])
        print()
        n+=1
print(n)


0 ['THAYER, HORTON, and BRODERICK, JJ., did not sit; BATCHELDER, J., retired, participated by special assignment under RSA 490:3; all who participated concurred.']
0 ['JOHNSON, J.']

2 ['THAYER, HORTON, and BRODERICK, JJ., did not sit; BATCHELDER, J., retired, participated by special assignment under RSA 490:3; all who participated concurred.']
2 ['PER CURIAM.']

3 ['All concurred.']
3 ['BRODERICK, J.']

5 ['BRODERICK, J., did not sit; the others concurred.']
5 ['THAYER, J.']

6 ['All concurred.']
6 ['HORTON, j.']

7 ['All concurred.']
7 ['BROCK, C.J.']

8 ['All concurred.']
8 ['BROCK, C.J.']

9 ['All concurred.']
9 ['JOHNSON, J.']

10 ['All concurred.']
10 ['HORTON, J.']

12 ['All concurred.']
12 ['BRODERICK, J.']

13 ['All concurred.']
13 ['HORTON, J.']

14 ['BRODERICK, J., did not sit; THAYER, J., sat but did not participate in the decision; JOHNSON, J., concurred; BROCK, C.J., dissented.']
14 ['HORTON, J.', 'BROCK, C.J.,']

15 ['BRODERICK, J., did not sit; the others concurred.']
1

149 ['THAYER, J., sat for oral argument but resigned prior to the final vote; BRODERICK, J., concurred; HORTON, J., retired, specially assigned under RSA 490:3, concurred.']
149 ['BROCK, C.J.']

150 ['BROCK, C.J., and BRODERICK and NADEAU, JJ., concurred; HORTON, J., retired, specially assigned under RSA 490:3, concurred; NADEAU and DALIANIS, JJ., took part in the final vote by consent of the parties. •']
150 ['DALIANIS, J.']

151 ['THAYER, J., sat for oral argument but resigned prior to the final vote; BROCK, C.J., concurred; NADEAU and DALIANIS, J J., who took part in the final vote by consent of the parties, concurred; HORTON, J., retired, specially assigned under RSA 490:3, concurred.']
151 ['BRODERICK, J.']

152 ['JOHNSON, J., sat for oral argument but retired prior to the final vote; THAYER, J., sat for oral argument but resigned prior to the final vote; BROCK, C.J., and BRODERICK, J., concurred.']
152 ['HORTON, J.']

153 ['Broderick and Nadeau, JJ., did not sit; Johnson, J., ret

230 ['THAYER, J., sat for oral argument but resigned prior to the final vote; BROCK, C.J., and BRODERICK and DALIANIS, JJ., concurred; HORTON, J., retired, specially assigned under RSA 490:3, concurred; NADEAU and DALIANIS, JJ., took part in the final vote by consent of the parties.']
230 ['NADEAU, J.']

231 ['BROCK, C.J., sat for oral argument but did not take part in the final vote; JOHNSON, J., sat for oral argument but retired prior to the final vote; BRODERICK, J., concurred; NADEAU and DALIANIS, JJ., who took part in the final vote by consent of the parties, concurred.']
231 ['HORTON, J.']

232 ['BROCK, C.J., sat for oral argument but did not take part in the final vote; JOHNSON, J., sat for oral argument but retired prior to the final vote; THAYER, J., sat for oral argument but resigned prior to the final vote; HORTON, BRODERICK, and DALIANIS, JJ., concurred; NADEAU and DALIANIS, JJ., took part in the final vote by consent of the parties.']
232 ['NADEAU, j.']

233 ['NADEAU, J., 

344 ['DALIANIS and DUGGAN, JJ., concurred; SMITH and HOLLMAN, JJ., superior court justices, specially assigned under RSA 490:3, concurred.']
344 ['NADEAU, J.']

345 ['BROCK, C.J., and BRODERICK and DUGGAN, JJ., concurred.']
345 ['DALIANIS, J.']

346 ['Brock, C.J., and Broderick, Nadeau and Duggan, jj., concurred.']
346 ['DALIANIS, J.']

347 ['BROCK, C.J., and BRODERICK, J., concurred; HORTON, J., retired, specially assigned under RSA 490:3, concurred; NADEAU, J., took part in the final vote by consent of the parties.']
347 ['NADEAU, J.']

348 ['GRAY and MANIAS, JJ., retired superior court justices, specially assigned under RSA 490:3, concurred.']
348 ['BARRY, j.,']

349 ['Brock, C.J., and Broderick, Dalianis and Duggan, JJ., concurred.']
349 ['NADEAU, J.']

350 ['BRODERICK and DAL1ANIS, JJ., concurred; HORTON, J., retired, and GROFF, J., superior court justice, specially assigned under RSA 490:3, concurred.']
350 ['NADEAU, J.']

351 ['DALIANIS and DUGGAN, JJ., concurred; MANGONES and M

1398 ['Chase, J.']

1399 ['Parsons, J., did not sit: the others concurred.']
1399 ['Blodgett, C. J.']

1400 ['Pike and Peaslee, JJ., did not sit; the others concurred.']
1400 ['Clark, C. J.']

1401 ['All concurred.']
1401 ['Parsons, J.']

1402 ['Clark, J., did not sit: the others concurred.']
1402 ['Wallace, J.']

1403 ['Blodgett, C. J., did not sit: the others concurred.']
1403 ['Parsons, J.']

1404 ['Parsons, J., did not sit: the others concurred.']
1404 ['Chase, J.']

1405 ['Blodgett, C. J., did not sit: the others concurred.']
1405 ['Peaslee, J.']

1406 ['Clark, J., did not sit: the others concurred.']
1406 ['Parsons, J.']

1407 ['All concurred.']
1407 ['Parsons, J.']

1408 ['All concurred.']
1408 ['Chase, J.']

1409 ['Chase and Peaslee, JJ., did not sit: the others concurred.']
1409 ['Wallace, J.']

1410 ['Parsons, J., did not sit: the others concurred.']
1410 ['Pike, J.']

1411 ['Clark, J., did not sit: the others concurred.']
1411 ['Wallace, J.']

1412 ['All concurred.']
1412 ['

...

17149 ['Foster, J., did not sit: the others concurred.']
17149 ['Allen, J.']

17150 ['Clark, J., did not sit: the others concurred.']
17150 ['Stanley, J.']

17151 ['Stanley, J., did not sit: the others concurred.']
17151 ['Allen, J.']

17152 ['Dob, C. J., did not sit: the others concurred.']
17152 ['Bingham, J.']

17153 ['Bingham and Smith, JJ., did not sit: the others concurred.']
17153 ['Stanley, J.']

17155 ['All concurred.']
17155 ['Bingham, J.']

17157 ['Stanley, J., did not sit: the others concurred.']
17157 ['Dok, C. J.']

17159 ['Stanley, J., did not sit: tbe others concurred.']
17159 ['Smith, J.']

17160 ['Bingham, J., did not sit: the others concurred.']
17160 ['Clark, J.']

17161 ['Bingham and Smith, JJ., did not sit: the others concurred.']
17161 ['Allen, J.']

17162 ['Smith, J., did not sit: the others concurred.']
17162 ['Stanley, J.']

17164 ['Doe, C. J., did not sit: the others concurred.']
17164 ['Clark, J.']

17165 ['Foster, J., did not sit: the others concurred.']
17

17359 ['All concurred.']
17359 ['Chase, J.']

17360 ['Chase, J., did not sit: the others concurred.']
17360 ['Blodgett, J.']

17361 ['Carpenter, J., did not sit: the others concurred.']
17361 ['Wallace, J.']

17362 ['Chase and Wallace, JJ., did not sit: the others concurred.']
17362 ['Smith, J.']

17363 ['Chase and Wallace, JJ., did not sit: the others concurred.']
17363 ['Parsons, J.']

17364 ['Wallace, J., did not sit: the others concurred.']
17364 ['Smith, J.']

17365 ['All concurred.']
17365 ['Parsons, J.']

17366 ['Wallace, J., did not sit: the others concurred.']
17366 ['Parsons, J.']

17367 ['All concurred.']
17367 ['Parsons, J.']

17368 ['Clark, Wallace, and Pike, JJ., dissented: the others concurred.']
17368 ['Carpenter, C. J.']

17369 ['All concurred.']
17369 ['Wallace, J.']

17370 ['Chase, J., did not sit: the others concurred.']
17370 ['Carpenter, J.']

17371 ['All concurred.']
17371 ['Wallace, J.']

17372 ['Blodgett, J., did not sit: the others concurred.']
17372 ['Wallace

17957 ['Young and Plummer, JJ., did not sit: the others concurred.']
17957 ['Parsons, C. J.']

17958 ['All concurred.']
17958 ['Parsons, C. J.']

17959 ['All concurred.']
17959 ['Parsons, C. J.']

17960 ['Plummer, J., did not sit: the others concurred.']
17960 ['Parsons, C. J.']

17961 ['All concurred.']
17961 ['Plummer, J.']

17962 ['All concurred.']
17962 ['Parsons, C. J.']

17963 ['All concurred.']
17963 ['Peaslee, J.']

17964 ['Frank N. Parsons. Reuben E. Walker. John E. Young. Robert J. Peaslee. William A. Plummer.']
17964 [None]

17965 ['All concurred.']
17965 ['Plummer, J.']

17966 ['All concurred.']
17966 ['Peaslee, J.']

17968 ['All concurred.']
17968 ['Peaslee, J.']

17969 ['All concurred.']
17969 ['Walker, J.']

17970 ['All concurred.']
17970 ['Peaslee, J.']

17971 ['Peaslee, J., did not sit: the others concurred.']
17971 ['Walker, J.']

17972 ['All concurred.']
17972 ['Young, J.']

17973 ['All concurred.']
17973 ['Peáslee, J.']

17974 ['•All concurred.']
17974 ['Walker, J.'

20502 ['BRODERICK, C.J.', 'DALIANIS, J.,']

20503 ['Dalianis, Duggan, Hicks and Conboy, JJ., concurred.']
20503 ['BRODERICK, C.J.']

20504 ['Dalianis, Duggan and Hicks, JJ., concurred.']
20504 ['BRODERICK, C.J.']

20505 ['Broderick, C.J., and Dalianis, Duggan and Conboy, JJ., concurred.']
20505 ['HICKS, J.']

20506 ['Broderick, C.J., and Dalianis and Duggan, JJ., concurred.']
20506 ['Hicks, J.']

20507 ['Dalianis, Duggan and Hicks, JJ., concurred.']
20507 ['BRODERICK, C.J.']

20508 ['Broderick, C.J., and Dalianis, Hicks and Conboy, JJ., concurred.']
20508 ['DUGGAN, J.']

20509 ['Broderick, C.J., and Duggan, Hicks and Conboy, JJ., concurred.']
20509 ['DALIANIS, J.']

20510 ['Broderick, C.J., and Duggan, Hicks and Conboy, JJ., concurred.']
20510 ['DALIANIS, J.']

20511 ['Broderick, C.J., and Duggan, Hicks and Conboy, JJ., concurred.']
20511 ['Dalianis, J.']

20512 ['DALIANIS and CONBOY, JJ., concurred; Broderick, C.J., dissented; Duggan, J., dissented.']
20512 ['HICKS, J.', 'BRODERICK, C

20762 ['All concurred.']
20762 ['Batchelder, J.']

20763 ['All concurred.']
20763 ['Batchelder, J.']

20765 ['Souter, J., concurred in the result; the others concurred.']
20765 ['King, C.J.']

20767 ['Batchelder, J., did not sit; Dickson, J., sat by special assignment pursuant to RSA 490:3; all concurred.']
20767 ['Douglas, J.']

20768 ['Brock and Batchelder, JJ., concurred in the result only; the others concurred.']
20768 ['DOUGLAS, J.']

20769 ['Douglas, J., dissented; the others concurred.']
20769 ['Batchelder, J.', 'Douglas, J.,']

20771 ['All concurred.']
20771 ['Souter, J.']

20772 ['King, C.J., and Souter, J., concurred specially.']
20772 ['Per CURIAM.', 'King, C.J.,', 'SOUTER, J.,']

20773 ['Souter, J., did not sit; the others concurred.']
20773 ['Batchelder, J.']

20774 ['Batchelder, J., did not sit; the others concurred.']
20774 ['Brock, J.']

20775 ['All concurred.']
20775 ['Souter, J.']

20776 ['All concurred.']
20776 ['Brock, J.']

20777 ['King, C.J., did not sit; the othe

In [95]:
# Finding cases with two authors, Blandin and Duncan
for i in range(len(cases_df)):
    if cases_df['opinion_author'][i]==['Blandin, J.', 'Duncan, J.']:
        print(i, cases_df['opinion_author'][i])
print(i)

# Finding cases with two authors, Duncan and Grimes
for i in range(len(cases_df)):
    if cases_df['opinion_author'][i]==['Duncan, J.', 'Grimes, J.']:
        print(i, cases_df['opinion_author'][i])
print(i)

4461 ['Duncan, J.', 'Grimes, J.']
8897 ['Duncan, J.', 'Grimes, J.']
13189


In [91]:
# This case 4051 has a dissenting opinion after the majority opinion
cases_df[4051:4052]

,citation_num,citation_type,court_id,court_name,decision_date,defendant,defendants_attorneys,docket_number,head_matter,id,judges,jurisdiction_id,jurisdiction_name,name,name_abbreviation,opinion_author,opinion_text,opinion_type,parties,plaintiff,plaintiff_attorneys,reporter
4051,95 N.H. 217,official,8797,New Hampshire Supreme Court,1948-07-06,Ellard,William H. Sleeper and Robert Shaw (Mr. Sleepe...,No. 3752,"No. 3752.\nState v. Leon O. Ellard.\nJuly 6, 1...",2352638,"[Duncan, J. dissented: the others concurred.]",47,New Hampshire,State v. Leon O. Ellard,State v. Ellard,"[Blandin, J., Duncan, J.]","[Blandin, J.\nThe respondent contends that the...","[majority, dissent]",[State v. Leon O. Ellard.],State,"Gordon M. Tiffany, Assistant Attorney General,...",New Hampshire Reports


In [88]:
# This case 4051 has a dissenting opinion after the majority opinion
cases_df['opinion_text'][4051]

["Blandin, J.\nThe respondent contends that the indictment was faulty, first, because it does not set out that the respondent was “the officer, agent, or servant of a corporation, public or private, or the clerk, servant or agent of a person within the meaning of R. L., c. 450, s. 28,” and secondly, he objects because it does not “fully and plainly, substantially and formally,” describe the crime of which he is accused. Both these exceptions are overruled. The State Liquor Commission created by R. L., c. 170, and for which the defendant worked, is in effect a State agency and a part of the State government. St. Regis Co. v. Board, 92 N. H. 164, 166, 167. The State itself is a body politic. Const., Pt. II, Art. 1. Under the provisions of R. L., c. 7, s. 9, the word person may “extend and be applied to bodies . . . politic as well as to individuals.” Dinnin v. Hutchins, 75 N. H. 470, 471. All monies and property in possession of the commission or its employees belong to the State which t

In [96]:
# This case 4461 has a dissenting opinion after the majority opinion
cases_df['opinion_text'][4461]

['Duncan, J.\nThe defendant was convicted of driving while under the influence of intoxicating liquor, following an accident which occurred in Nashua on March 12, 1970 at approximately 1:42 A.M. In the course of the trial he moved to strike the testimony of the arresting officer that die defendant had admitted that he was the driver of the vehicle, upon tire ground that he was given no prior warning of his right to remain silent. The motion was denied subject to defendant’s exception, as was his motion to suppress the evidence made at die close of the State’s evidence.\nThe “ question of law raised by the defendant’s motion for suppression ” of die evidence was reserved and transferred by the Justice of the district court ( Guertin, J. ).\nThe testimony at the hearing in the district court was not recorded. See State v. Wildman, 110 N.H. 340, 267 A.2d 575 (1970 ). The reserved case states that no evidence beyond the testimony of the arresting officer was introduced to show that the def

In [99]:
cases_df[cases_df['docket_number']=='No. 2014-0812']

,citation_num,citation_type,court_id,court_name,decision_date,defendant,defendants_attorneys,docket_number,head_matter,id,judges,jurisdiction_id,jurisdiction_name,name,name_abbreviation,opinion_author,opinion_text,opinion_type,parties,plaintiff,plaintiff_attorneys,reporter
12767,168 N.H. 694,official,8797,New Hampshire Supreme Court,2016-04-01,NaN,"Shaheen & Gordon, P.A., of Manchester (Jared O...",No. 2014-0812,No. 2014-0812\nIn the Matter of Holly Doherty ...,6789703,"[Dalianis, C.J., and CONBOY, J., concurred.]",47,New Hampshire,In the Matter of Holly Doherty and William Doh...,In re Doherty,"[BASSETT, J.]","[BASSETT, J.\nThe 10th Circuit Court-Brentwood...",[majority],[In the Matter of Holly Doherty and William Do...,In re Doherty,"Primmer Piper Eggleston & Cramer PC, of Manche...",New Hampshire Reports


In [102]:
cases_df.opinion_text[12770]

['HICKS, J.\nFarmington School District (district) appeals a decision of the State Board of Education (state board) reversing the decision of the Farmington School Board (local board) not to renew the employment contract of Demetria McKaig, a guidance counselor at Farmington High School. McKaig cross-appeals the state board’s decision and argues that she is entitled to reinstatement with back pay and benefits. We affirm in part, reverse in part, order that McKaig be reinstated to her former employment, and remand to the board for further proceedings to determine whether she is entitled to additional remedies.\nThe state board’s decision includes the following facts. McKaig was a high school guidance counselor employed by the district. In November 2012, a student — whom, consistent with the record, we will refer to as “Student A” — and her boyfriend told McKaig and another guidance counselor that Student A was pregnant and that she wanted to terminate her pregnancy. Student A was fiftee

In [28]:
print(''.join(cases_df['opinion_text'][20]))

THAYER, J.
Justin D., a juvenile (Justin), appeals his adjudication of delinquency following a bench trial at the Concord District Court {Korbey, J.) for possessing a dangerous weapon at the time of his arrest in violation of RSA 159:15 (1994) (amended 1998).We affirm.
The following facts were adduced at trial. On February 21, 1997, sixteen-year-oíd Justin and the victim agreed to fight. The fight occurred on Woodman Street, behind Concord High School. The Concord Police Department was called to respond to the fight. Officer Reilly was the first officer to arrive on the scene. He did not witness the fight but saw the victim lying in the street, receiving CPR. The victim appeared to be “hurt very badly.” He stopped breathing and died the following morning from a subdural hematoma caused by a blunt force to the head.
A student informed Officer Reilly that Justin was involved in the incident and that he was inside the school. Officer Reilly found Justin in the vice principal’s office and 

Affirmed.


In [48]:
# To read full opinion text change the number to the desired index number (called case# below)

cases_df['opinion_text'][19][0]


'JOHNSON, J.\nThe employer, the Rockingham County Sheriff’s Department, appeals a decision of the compensation appeals board (board) finding that the claimant, Michael Alden, suffered a work-related injury and was entitled to workers’ compensation benefits. We reverse and remand.\nThe record reveals the following facts. The claimant testified that he was employed for approximately nineteen years as a deputy sheriff with the Rockingham County Sheriff’s Department. He testified that on March 5, 1997, he injured his back while moving a heavy credenza in an office in the sheriff’s department. The claimant then asked a co-worker for a workers’ compensation claim form and immediately left the sheriff’s office. He has never returned to work. The employer denied the claimant’s claim for benefits on the basis that there was no causal relationship between his injury and his employment. The claimant petitioned the department of labor, and the hearing officer denied benefits. See RSA 281-A:43, 1(a

In [29]:
# cases_df[17:20]
pd.set_option('display.max_rows', 60)

cases_df['decision'].value_counts()


[Affirmed]                                                                                                         2617
[Exceptions overruled]                                                                                             1234
[Remanded]                                                                                                          822
[Reversed and remanded]                                                                                             819
[Exception overruled]                                                                                               657
[Case discharged]                                                                                                   553
[Reversed]                                                                                                          404
[Judgment on the verdict]                                                                                           306
[New trial]                             

In [46]:
cases_df['decision'][0]

['So ordered']

In [21]:
cases_df['decision'][395:405]
# cases_df['decision'][12995:13004]

# df.at[13003, 'decision'] = 'test'

395                                           [Reversed]
396    [Reversed in part; vacated in part; and remanded]
397                                           [Reversed]
398                                           [Affirmed]
399                                           [Affirmed]
400                                           [Affirmed]
401                 [Affirmed in part; reversed in part]
402                                           [Remanded]
403                                           [Affirmed]
404                              [Reversed and remanded]
Name: decision, dtype: object

In [20]:
cases_df = cases_df[0:13189]

In [21]:
# %cd "C:\\Users\\Dolomite\\Desktop\\Analytics\\Riverstone\\New_Hampshire"
# cases_df.to_csv('cases_df1900.csv', sep=',')

C:\Users\Dolomite\Desktop\Analytics\Riverstone\New_Hampshire


In [65]:
# cases_df['decision'].value_counts()
# cases_df['decision'][12800:13010]
# len(cases_df)
cases_df['opinion_text'][13007]

["Kenison, C.J.\nThe issue in this case is whether the trial court abused his discretion in refusing to permit plaintiff to introduce into evidence safety standards promulgated by the National Safety Council (1964).\nThe case arose as a result of a fall sustained on October 17, 1966, by Evelyn Lemery on department store premises owned and controlled by defendant, O’Shea Dennis Corporation. While leaving the store after shopping there, plaintiff fell from a step-down at the end of an exit ramp which sloped from the store to the sidewalk. The ramp in question was eight feet and seven inches in length terminating at the step-down to the sidewalk. The step-down was four and a half to five inches in height' as measured from the sidewalk to the top of the ramp. The evidence shows that plaintiff was not carrying any packages, that the day was clear and dry, and that she knew of the ramp-step-down exit arrangement from prior use. She stated her belief that she had fallen because she had “mis-j

In [7]:
# To find citations, look for the word 'see' followed by a 4 digit number with closed parenthesis ')'.
# This captures a string terminated by the pattern '(year)'. Exclude the opening parenthesis due to variations in the pattern
# look for the word 'also' as there can be back to back citations, eg. See citation 1 (2000) also citation 2 (1966)
# a third instance of citation does not use either 'see' or 'also'. This occurs generally when many citations are listed

## THERE ARE STILL SEVERAL INSTANCES NOT OBTAINED WITH THIS SEARCH, eg. 'Ricci, 144 N.H. at 243, 739 A.2d at 406.' and ' See N.H. Const. pt. I, art. 19; U.S. Const. amend. IV'
list_of_citations=[]

# for the first i cases in the parsed data frame
for i in range(20):
    
   # find phrases starting with 'see', case-insensitive
   list_see_citations = (re.findall("see\s.*\d{4}\)", cases_df['opinion_text'][i][0], flags=re.IGNORECASE))

   # find phrases with 'also', case-insensitive
   list_also_citations = (re.findall("[^s].also\s.*\d{4}\)", cases_df['opinion_text'][i][0], flags=re.IGNORECASE))

   # find phrases that have the 'v.' and end in a year with format 'XXXX)', it also grabs up to 50 characters preceding the 'v.', or a period, whichever comes first
   list_versus_citations = (re.findall("\.+\s.{0,50}[v]\..{0,100}[0-9]{4}\)", cases_df['opinion_text'][i][0], flags=re.IGNORECASE|re.MULTILINE))

   if len(list_see_citations)!=0:
       print(i, list_see_citations)
       print(i, list_also_citations)
       print(i, list_versus_citations)
       print()
       print()
       list_of_citations.append([i,list_see_citations, list_also_citations])
   else:
       pass

0 ['see Wehringer’s Case, 130 N.H. 707, 716, 547 A.2d 252, 257 (1988), we retain the ultimate authority to determine whether, on the facts found, a violation of the rules governing attorney conduct has occurred and, if so, the appropriate sanction. Wood’s Case, 137 N.H. 698, 701, 634 A.2d 1340, 1342 (1993)', 'See RSA ch. 570-A (1986 & Supp. 1998) (amended 1998)', 'See Appeal of Trotzer, 143 N.H. 64, 68-69, 719 A.2d 584, 587-88 (1998)']
0 []
0 []


1 ['See State v. Dellorfano, 128 N.H. 628, 632, 517 A.2d 1163, 1166 (1986) (claim limited to Federal Constitution where State Constitution not specifically invoked on appeal). In Crane, the prosecution relied almost entirely on the defendant’s confession to secure a conviction, and the defendant sought to introduce evidence of the circumstances surrounding it to undermine its reliability. Crane, 476 U.S. at 685. Here, however, the defendant’s statement was never introduced so that evidence of the circumstances surrounding it were not necessar

6 ['see RSA 629:1 (1996); RSA 632-A:2 (1996) (amended 1997, 1998)', 'See RSA 682-A:4 (1996), :1, IV (1996) (amended 1998)', 'see State v. Bean, 117 N.H. 185, 187, 371 A.2d 1152, 1153 (1977), that is considered a substantive offense in and of itself, see State v. Harper, 126 N.H. 815, 818, 498 A.2d 310, 313 (1985)', 'See RSA 629:1, I; State v. Allen, 128 N.H. 390, 392-96, 514 A.2d 1263, 1265-68 (1986) (despite statutory variants of murder, an indictment for the offense of attempted murder may plead the intended crime generally). “It is ordinarily sufficient to state the intended offense generally, as by alleging an intent to steal, or commit the crime of larceny, rape, or arson.” People v. Williams, 288 N.E.2d 406, 409 (Ill. 1972) (quotation omitted); see Allen, 128 N.H. at 392-96, 514 A.2d at 1265-68; see also 2 C. TORCIA, WHARTON’S CRIMINAL PROCEDURE § 261, at 121 (13th ed. 1990); State v. Chaisson, 123 N.H. 17, 24, 458 A.2d 95, 99 (1983). Since an attempted crime is by definition a c

13 ['see RSA 636:1 (1997), and two counts of theft by unauthorized taking, see RSA 637:3 (1997)', 'see Miranda v. Arizona, 384 U.S. 436 (1966)', 'See State v. Beland, 138 N.H. 735, 737, 645 A.2d 79, 80 (1994); Colorado v. Connelly, 479 U.S. 157, 158 (1986)', 'see State v. Jenner, 451 N.W.2d 710, 716 (S.D. 1990), cert. denied, 510 U.S. 822 (1993), and we would not reach a different result here even if that rule were adopted. A finding of voluntariness, based ultimately on the fact finder’s comprehension of the totality of the facts, cf. Commissioner v. Duberstein, 363 U.S. 278, 289 (1960), relies on the unique position of the fact finder, who assesses first-hand all of the verbal and nonverbal aspects of evidence presented, cf. State v. Little, 138 N.H. 657, 661, 645 A.2d 665, 667 (1994). Words printed on the sterile pages of a transcript do not convey the intangible dynamics or full sensory experience of trial that may influence evaluation of the facts. Cf. id. Hence, while the legal s

16 ['h also refers to the specific insured being sued or to any insured under the policy. It is reasonable to assume that while “the Insured” in certain policy provisions may refer to different persons from “an Insured” in other provisions, cf. Pawtucket Mut. Ins. Co. v. Lebrecht, 104 N.H. 465, 468, 190 A.2d 420, 422 (1963), it refers to the same persons as “the Insured” in other provisions unless defined otherwise, see Dupont v. Chagnon, 119 N.H. 792, 794, 408 A.2d 408, 409 (1979). In this case, because a reasonable reading of the notice provisions could give rise to different interpretations, we find the term “the Insured” ambiguous. See Whitcomb v. Peerless Ins. Co., 141 N.H. 149, 151, 679 A.2d 575, 577 (1996). Construing the ambiguity in favor of coverage, see Weaver v. Royal Ins. Co. of America, 140 N.H. 780, 781-82, 674 A.2d 975, 977 (1996)', 'e also Zuckerman v. Nat. Union Fire Ins., 495 A.2d 395, 406 (N.J. 1985). Claims-made policies necessarily include a presumption that the i